# Train Final Action Prediction Network

In [1]:
import numpy as np
import os
import json
import torch
import torch.nn as nn

In [2]:
train_file = '.\\task1_classic_classification\\task1_classic_classification\\calms21_task1_train.json'
with open(train_file, 'r') as file:
    train_data = json.load(file)
test_file = '.\\task1_classic_classification\\task1_classic_classification\\calms21_task1_test.json'
with open(test_file, 'r') as file:
    test_data = json.load(file)

FileNotFoundError: [Errno 2] No such file or directory: '.\\task1_classic_classification\\task1_classic_classification\\calms21_task1_train.json'

In [7]:
def chunk(data):
    new_data = []
    for vid in data:
        # split into 150 frame pieces
        if vid.shape[0] > 160:
            section_size = 10
            num_splits = vid.shape[0] // section_size
            elements_to_keep = section_size * num_splits
            truncated_vid = vid[:elements_to_keep]
            new_data += np.split(truncated_vid, num_splits)
    return np.array(new_data)

In [23]:
# if 50% of a window has an action then count it as worthy
# {'attack': 0, 'investigation': 1, 'mount': 2, 'other': 3}
def reduce_labels(data):
    reduced_data = []
    action_percent = 0.5
    for clip in data:
        if (clip < 3).sum() > clip.size*action_percent:
            reduced_data.append(1)
        else:
            reduced_data.append(0)
    return np.array(reduced_data)

In [84]:
# label 1 means there is an action
train_x = chunk([np.array(video['keypoints']) for video in train_data['annotator-id_0'].values()])
train_y = (chunk([np.array(video['annotations']) for video in train_data['annotator-id_0'].values()]))
test_x = chunk([np.array(video['keypoints']) for video in test_data['annotator-id_0'].values()])
test_y = (chunk([np.array(video['annotations']) for video in test_data['annotator-id_0'].values()]))

In [25]:
train_x = train_x.reshape(train_x.shape[0], -1)
test_x = test_x.reshape(test_x.shape[0], -1)

In [85]:
def get_stats(data):
    total = data.size
    return [(data==i).sum()/total for i in range(4)]
get_stats(train_y)

[np.float64(0.02752626288507401),
 np.float64(0.2888877939176538),
 np.float64(0.05635926444212311),
 np.float64(0.6272266787551491)]

In [77]:
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        depth = 3
        layers = [
            nn.Linear(input_size, hidden_size),
            nn.ReLU()]
        for i in range(depth):
            layers.append(nn.Linear(hidden_size, hidden_size)),
            layers.append(nn.ReLU())
        layers.append(nn.Linear(hidden_size, output_size))
        self.net = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.net(x)

In [78]:
num_epochs = 1
print_every = 100
input_size = 10*2*2*7
hidden_size = 1000
output_size = 1
model = SimpleNN(input_size, hidden_size, output_size)

optim = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
loss_fcn = nn.MSELoss()

In [79]:
train_x = torch.tensor(train_x, dtype=torch.float32)
train_y = torch.tensor(train_y, dtype=torch.float32)
test_x = torch.tensor(test_x, dtype=torch.float32)
test_y = torch.tensor(test_y, dtype=torch.float32)

C:\Users\audre\AppData\Local\Temp\ipykernel_22452\2581623885.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_x = torch.tensor(train_x, dtype=torch.float32)
C:\Users\audre\AppData\Local\Temp\ipykernel_22452\2581623885.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_y = torch.tensor(train_y, dtype=torch.float32)
C:\Users\audre\AppData\Local\Temp\ipykernel_22452\2581623885.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_x = torch.tensor(test_x, dtype=torch.float32)
C:\Users\audre\AppData\Local\Temp\ipykernel_2245

In [80]:
model

SimpleNN(
  (net): Sequential(
    (0): Linear(in_features=280, out_features=1000, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1000, out_features=1000, bias=True)
    (3): ReLU()
    (4): Linear(in_features=1000, out_features=1000, bias=True)
    (5): ReLU()
    (6): Linear(in_features=1000, out_features=1000, bias=True)
    (7): ReLU()
    (8): Linear(in_features=1000, out_features=1, bias=True)
  )
)

In [81]:
# preds = model.forward(test_x)
def accuracy(preds, real):
    return ((preds > 0.5).int().flatten() == real).sum() / test_y.shape[0]

In [82]:
# train
for epoch in range(num_epochs):
    for iter in range(train_x.shape[0]):
        inputs = train_x[iter]
        labels = train_y[iter]
        outputs = model(inputs)
        loss = loss_fcn(outputs, labels)

        loss.backward()
        optim.step()
        optim.zero_grad()

        if iter % print_every == 0:
            preds = model.forward(test_x)
            test_loss = loss_fcn(test_y, preds)
            acc = accuracy(preds, test_y)
            # logging
            print("iter {}, train_loss {:.4e}, test_loss {:.4e}".format(iter, loss.item(), test_loss.item()))
            print("accuracy", acc)

c:\Users\audre\anaconda3\envs\mouse\Lib\site-packages\torch\nn\modules\loss.py:634: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\audre\anaconda3\envs\mouse\Lib\site-packages\torch\nn\modules\loss.py:634: UserWarning: Using a target size (torch.Size([26203, 1])) that is different to the input size (torch.Size([26203])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


iter 0, train_loss 7.6030e-01, test_loss 1.0232e+03
accuracy tensor(0.4003)
iter 100, train_loss 9.1046e-02, test_loss 1.0449e+01
accuracy tensor(0.5170)
iter 200, train_loss 1.9693e+00, test_loss 2.8038e+01
accuracy tensor(0.4837)
iter 300, train_loss 2.9931e+00, test_loss 3.5303e+01
accuracy tensor(0.4002)
iter 400, train_loss 1.5055e+00, test_loss 4.1165e+00
accuracy tensor(0.4722)
iter 500, train_loss 2.8877e-01, test_loss 6.6426e+00
accuracy tensor(0.6063)
iter 600, train_loss 3.3489e-03, test_loss 7.7160e+00
accuracy tensor(0.5909)
iter 700, train_loss 3.3115e-02, test_loss 8.3213e+00
accuracy tensor(0.3927)
iter 800, train_loss 3.8180e-02, test_loss 1.7467e+01
accuracy tensor(0.3957)
iter 900, train_loss 2.0368e-02, test_loss 1.3839e+01
accuracy tensor(0.5973)
iter 1000, train_loss 6.4135e-04, test_loss 2.1734e+00
accuracy tensor(0.4914)
iter 1100, train_loss 6.5341e-02, test_loss 2.0716e+00
accuracy tensor(0.5997)
iter 1200, train_loss 7.9724e-01, test_loss 6.2251e-01
accuracy 

KeyboardInterrupt: 

In [13]:
data['annotator-id_0']['task1/test/mouse071_task1_annotator1'].keys()

dict_keys(['annotations', 'keypoints', 'metadata', 'scores'])

In [ ]:
def load_keypoints_test_train():
    prefix = '.\\task1_MARS_features\\calms21_task1_features'
    train_files = os.listdir(prefix + '\\train')
    test_files = os.listdir(prefix + '\\test')
    test, train = [], []
    for file in train_files:
        train.append(np.load(prefix + '\\train\\' + file))
    for file in test_files:
        test.append(np.load(prefix + '\\test\\' + file))
    return train, test

In [36]:
train_keypoints, test_keypoints = load_keypoints_test_train()

In [37]:
train[0]['keypoints'].shape
# 30 fps, ~10 minutes
# 21364 frames
# 2 mice, xy pos, 7 features
# will probably look at ~5 seconds=150 frames

(21364, 2, 2, 7)